# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [22]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [23]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)


/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [24]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

            
#print(len(full_data_rows_list))
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [25]:
# check the number of rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [26]:
# make a connection to local machine instance of Cassandra 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# establish connection
session = cluster.connect()

#### Create Keyspace

In [27]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [28]:
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)

## Necessary queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### First query

In [29]:
#Creation of table for query 1.
#Primary key with sessionID as partition key and item_number_session as clustering column
query = "CREATE TABLE IF NOT EXISTS music_library_session "
query = query + "(sessionId int, item_number_session int, artist varchar, length_song float,\
song_title varchar, PRIMARY KEY(sessionId, item_number_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_session (sessionId, item_number_session, artist, length_song, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

In [31]:
#query1: artist, song title and song's length heard in sessionId = 338, itemInSession = 4
query = "select artist, song_title, length_song  from music_library_session WHERE sessionId=338 and item_number_session=4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length_song)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Second query

In [32]:
#Creation of table for query 2.
#userId and sessionId as composite partition key and item_number_session as clustering column.
query = "CREATE TABLE IF NOT EXISTS music_library_sessionUser "
query = query + "(userId int, sessionId int, item_number_session int, artist varchar, firstName_user varchar, \
lastName_user varchar, song_title varchar, PRIMARY KEY( (userId, sessionId), item_number_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_sessionUser (userId, sessionId, item_number_session, artist, firstName_user, \
        lastName_user, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

In [34]:
#query2: artist, song (sorted by itemInSession) and user names for userid=10 and sessionid=182
query = "select artist, song_title, firstName_user as fname, lastName_user as lname from music_library_sessionUser\
        WHERE sessionId=182 and userId=10;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.fname, row.lname)


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Third query

In [35]:
#Creation of table for query 3.
#song_title as primary key and userId as clustering column
query = "CREATE TABLE IF NOT EXISTS music_library_user "
query = query + "(song_title varchar, userId int, firstName_user varchar, lastName_user varchar,\
        PRIMARY KEY(song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [36]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_user (song_title, userId, firstName_user, lastName_user)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [37]:
#query3: user names who listened to the song 'All Hands Against His Own'
query = "select firstName_user as fname, lastName_user as lname from music_library_user WHERE song_title='All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.fname, row.lname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [38]:
query = "drop table music_library_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [39]:
query = "drop table music_library_sessionUser"
try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

In [40]:
query = "drop table music_library_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [41]:
session.shutdown()
cluster.shutdown()